# Gold Price Prediction with LSTM (End-to-End)

This notebook builds a complete, production-ready pipeline to predict daily gold prices using Long Short-Term Memory (LSTM) neural networks. It includes:

- **Data Collection**: Using `scripts/download_data.py` to fetch and save raw data
- **Data Preprocessing**: Feature engineering, scaling, and saving preprocessed data
- **Model Training**: Using `scripts/train_lstm.py` for training with proper model persistence
- **Model Evaluation**: Computing metrics and saving results to CSV
- **Visualization**: Creating publication-ready plots saved to `data/processed/plots/`
- **Future Predictions**: Generating 7-day forecasts using `scripts/run_prediction.py`

This notebook follows best practices:
- Uses relative imports from scripts/
- Saves all intermediate results to appropriate folders
- Uses os.path.join() or pathlib.Path for all file paths
- Includes error handling and progress messages
- Sets random seed (42) for reproducibility


In [ ]:
## Setup and Imports

Set random seeds for reproducibility, import required libraries, configure paths, and add utility functions.

In [ ]:
import os
import sys
import warnings
from pathlib import Path
from datetime import datetime

warnings.filterwarnings("ignore")

# Print progress helper
def log(msg: str) -> None:
    print(f"[INFO] {msg}")

# Add scripts directory to path for imports
NOTEBOOK_DIR = Path.cwd() if "__file__" not in globals() else Path(__file__).parent
PROJECT_ROOT = NOTEBOOK_DIR.parent
SCRIPTS_DIR = PROJECT_ROOT / "scripts"
sys.path.insert(0, str(SCRIPTS_DIR))

# Reproducibility
import random
import numpy as np

random.seed(42)
np.random.seed(42)

try:
    import tensorflow as tf
    tf.keras.utils.set_random_seed(42)
    tf.config.experimental.enable_op_determinism()
except Exception as e:
    log("TensorFlow determinism not fully enabled; continuing.")

# Core libraries
import pandas as pd
import joblib
from sklearn.preprocessing import MinMaxScaler

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["axes.grid"] = True
sns.set_theme(style="whitegrid")

# Import from scripts
from download_data import main as download_data_main
from preprocess_data import (
    preprocess_gold_data,
    split_and_create_sequences,
    save_splits
)
from train_lstm import (
    set_random_seed,
    train_lstm_model,
    save_model_with_timestamp
)
from run_prediction import (
    predict_future,
    make_predictions,
    load_model_and_scaler
)
from evaluate_model import (
    evaluate_model,
    save_evaluation_results,
    print_metrics_table
)
from visualize_results import (
    plot_training_history,
    plot_actual_vs_predicted,
    plot_error_distribution,
    plot_feature_correlation,
    plot_last_n_days_with_confidence,
    plot_future_predictions
)

# Define paths using pathlib
BASE_DIR = PROJECT_ROOT
DATA_RAW_DIR = BASE_DIR / "data" / "raw"
DATA_PROCESSED_DIR = BASE_DIR / "data" / "processed"
DATA_PROCESSED_TRAIN_DIR = DATA_PROCESSED_DIR / "train"
DATA_PROCESSED_VAL_DIR = DATA_PROCESSED_DIR / "val"
DATA_PROCESSED_TEST_DIR = DATA_PROCESSED_DIR / "test"
DATA_PROCESSED_PLOTS_DIR = DATA_PROCESSED_DIR / "plots"
MODELS_DIR = BASE_DIR / "models"

# Create directories if they don't exist
for dir_path in [DATA_RAW_DIR, DATA_PROCESSED_DIR, DATA_PROCESSED_TRAIN_DIR,
                 DATA_PROCESSED_VAL_DIR, DATA_PROCESSED_TEST_DIR,
                 DATA_PROCESSED_PLOTS_DIR, MODELS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# File paths
RAW_DATA_PATH = DATA_RAW_DIR / "gold_raw_data.csv"
PREPROCESSED_DATA_PATH = DATA_PROCESSED_DIR / "gold_preprocessed_data.csv"
EVALUATION_RESULTS_PATH = MODELS_DIR / "evaluation_results.csv"
FUTURE_PREDICTIONS_PATH = DATA_PROCESSED_DIR / "future_predictions.csv"

log("Environment initialized.")
log(f"Project root: {BASE_DIR}")
log(f"Data raw directory: {DATA_RAW_DIR}")
log(f"Data processed directory: {DATA_PROCESSED_DIR}")
log(f"Models directory: {MODELS_DIR}")


In [ ]:
## Step 1: Data Collection

Download historical gold price data and economic indicators using `scripts/download_data.py`. The script will:
- Download 10 years of data for Gold Futures (`GC=F`) and economic indicators
- Download CPI from FRED (`CPIAUCSL`)
- Merge all datasets and save to `data/raw/gold_raw_data.csv`

If the data already exists, we'll load it from the file instead of downloading again.

# Data Collection: Use download_data.py script or load existing data
try:
    if RAW_DATA_PATH.exists():
        log(f"Raw data already exists at {RAW_DATA_PATH}. Loading...")
        raw_data = pd.read_csv(RAW_DATA_PATH, index_col=0, parse_dates=True)
        log(f"Loaded raw data: {raw_data.shape}")
    else:
        log("Raw data not found. Downloading...")
        # Run download script
        import subprocess
        result = subprocess.run(
            ["python", str(SCRIPTS_DIR / "download_data.py"),
             "--output-root", str(DATA_RAW_DIR)],
            capture_output=True,
            text=True,
            cwd=str(PROJECT_ROOT)
        )
        if result.returncode == 0:
            log("Download completed successfully.")
            raw_data = pd.read_csv(RAW_DATA_PATH, index_col=0, parse_dates=True)
        else:
            log(f"Download script error: {result.stderr}")
            raise RuntimeError("Data download failed")
    
    log(f"Raw data shape: {raw_data.shape}")
    log(f"Raw data columns: {raw_data.columns.tolist()}")
    log(f"Date range: {raw_data.index.min()} to {raw_data.index.max()}")
    
    # Display sample
    display(raw_data.head())
    display(raw_data.tail())
    
except Exception as e:
    log(f"Data collection failed: {e}")
    raise



In [ ]:
## Step 2: Data Preprocessing and Feature Engineering

Preprocess the raw data using `scripts/preprocess_data.py` functions:
- Create technical indicators (MA, RSI, returns, volatility)
- Add lag features for all numeric columns
- Scale features to [0, 1] using MinMaxScaler
- Save preprocessed data to `data/processed/gold_preprocessed_data.csv`
- Split data chronologically and create LSTM sequences
- Save train/val/test splits to their respective folders

# Data Preprocessing: Feature engineering and scaling
try:
    log("Starting preprocessing...")
    
    # Preprocess data
    preprocessed_data, feature_scaler = preprocess_gold_data(
        raw_data,
        target_col="Gold",
        lags=[1, 3, 5, 7, 10]
    )
    
    log(f"Preprocessed data shape: {preprocessed_data.shape}")
    
    # Save preprocessed data
    preprocessed_data.to_csv(PREPROCESSED_DATA_PATH)
    log(f"Preprocessed data saved to {PREPROCESSED_DATA_PATH}")
    
    # Display sample
    display(preprocessed_data.head())
    display(preprocessed_data.tail())
    
    # Split and create sequences
    SEQ_LEN = 60
    log(f"Creating sequences with length {SEQ_LEN}...")
    
    splits = split_and_create_sequences(
        preprocessed_data,
        target_col="Gold",
        seq_len=SEQ_LEN,
        train_ratio=0.70,
        val_ratio=0.15
    )
    
    # Extract sequences
    X_train, y_train, train_dates = splits["train"]
    X_val, y_val, val_dates = splits["val"]
    X_test, y_test, test_dates = splits["test"]
    
    # Save splits
    save_splits(preprocessed_data, splits, DATA_PROCESSED_DIR, target_col="Gold")
    
    log("Preprocessing complete!")
    
except Exception as e:
    log(f"Preprocessing failed: {e}")
    raise


In [ ]:
## Step 3: Model Training

Train the LSTM model using `scripts/train_lstm.py`:
- Build two-layer LSTM with dropout
- Train with early stopping
- Save model and scaler with timestamp to `models/` directory

# Model Training: Build and train LSTM
try:
    log("Starting model training...")
    
    # Train model
    model, history = train_lstm_model(
        X_train, y_train,
        X_val, y_val,
        epochs=100,
        batch_size=32,
        patience=10,
        lstm_units=50,
        dropout_rate=0.2,
        dense_units=25,
        verbose=1
    )
    
    # Save model and scaler with timestamp
    model_path, scaler_path = save_model_with_timestamp(
        model,
        feature_scaler,
        MODELS_DIR,
        prefix="gold_lstm"
    )
    
    log("Model training complete!")
    
except Exception as e:
    log(f"Model training failed: {e}")
    raise


In [ ]:
## Step 4: Model Evaluation

Evaluate the model on train, validation, and test sets:
- Make predictions on all splits
- Inverse transform predictions back to original scale
- Compute metrics (RMSE, MAE, MAPE, R²)
- Save evaluation results to `models/evaluation_results.csv`

# Model Evaluation: Compute metrics and save results
try:
    log("Starting model evaluation...")
    
    # Find target column index for inverse transform
    target_col_idx = list(preprocessed_data.columns).index("Gold")
    
    # Make predictions
    y_train_pred_scaled = model.predict(X_train, verbose=0).squeeze()
    y_val_pred_scaled = model.predict(X_val, verbose=0).squeeze()
    y_test_pred_scaled = model.predict(X_test, verbose=0).squeeze()
    
    # Inverse transform using the feature scaler
    # Create dummy arrays with target at correct position
    n_train = len(y_train)
    n_val = len(y_val)
    n_test = len(y_test)
    n_features = preprocessed_data.shape[1]
    
    # Inverse transform true values
    train_true_full = np.zeros((n_train, n_features))
    train_true_full[:, target_col_idx] = y_train
    y_train_true_inv = feature_scaler.inverse_transform(train_true_full)[:, target_col_idx]
    
    val_true_full = np.zeros((n_val, n_features))
    val_true_full[:, target_col_idx] = y_val
    y_val_true_inv = feature_scaler.inverse_transform(val_true_full)[:, target_col_idx]
    
    test_true_full = np.zeros((n_test, n_features))
    test_true_full[:, target_col_idx] = y_test
    y_test_true_inv = feature_scaler.inverse_transform(test_true_full)[:, target_col_idx]
    
    # Inverse transform predictions
    train_pred_full = np.zeros((n_train, n_features))
    train_pred_full[:, target_col_idx] = y_train_pred_scaled
    y_train_pred_inv = feature_scaler.inverse_transform(train_pred_full)[:, target_col_idx]
    
    val_pred_full = np.zeros((n_val, n_features))
    val_pred_full[:, target_col_idx] = y_val_pred_scaled
    y_val_pred_inv = feature_scaler.inverse_transform(val_pred_full)[:, target_col_idx]
    
    test_pred_full = np.zeros((n_test, n_features))
    test_pred_full[:, target_col_idx] = y_test_pred_scaled
    y_test_pred_inv = feature_scaler.inverse_transform(test_pred_full)[:, target_col_idx]
    
    # Evaluate model
    results_df = evaluate_model(
        y_train_true_inv, y_train_pred_inv,
        y_val_true_inv, y_val_pred_inv,
        y_test_true_inv, y_test_pred_inv
    )
    
    # Print metrics
    print_metrics_table(results_df)
    
    # Save evaluation results
    save_evaluation_results(results_df, EVALUATION_RESULTS_PATH, append=False)
    
    log("Evaluation complete!")
    
except Exception as e:
    log(f"Evaluation failed: {e}")
    raise


In [ ]:
## Step 5: Visualization

Create publication-ready visualizations and save them to `data/processed/plots/`:
1. Training/validation loss curves
2. Actual vs Predicted prices for train/val/test
3. Prediction error distribution
4. Feature correlation heatmap
5. Last 100 days with confidence bands

# Visualization: Create and save all plots
try:
    log("Creating visualizations...")
    
    # 1. Training history
    plot_training_history(
        history,
        output_path=DATA_PROCESSED_PLOTS_DIR / "training_history.png"
    )
    
    # 2. Actual vs Predicted
    plot_actual_vs_predicted(
        train_dates, y_train_true_inv, y_train_pred_inv,
        val_dates, y_val_true_inv, y_val_pred_inv,
        test_dates, y_test_true_inv, y_test_pred_inv,
        output_path=DATA_PROCESSED_PLOTS_DIR / "actual_vs_predicted.png"
    )
    
    # 3. Error distribution
    test_errors = y_test_true_inv - y_test_pred_inv
    plot_error_distribution(
        test_errors,
        split_name="Test",
        output_path=DATA_PROCESSED_PLOTS_DIR / "error_distribution.png"
    )
    
    # 4. Feature correlation heatmap
    plot_feature_correlation(
        preprocessed_data,
        output_path=DATA_PROCESSED_PLOTS_DIR / "feature_correlation.png"
    )
    
    # 5. Last 100 days with confidence bands
    plot_last_n_days_with_confidence(
        test_dates, y_test_true_inv, y_test_pred_inv,
        n_days=100,
        output_path=DATA_PROCESSED_PLOTS_DIR / "last_100_days_confidence.png"
    )
    
    log("Visualizations complete!")
    
except Exception as e:
    log(f"Visualization failed: {e}")
    raise


In [ ]:
## Step 6: Future Predictions

Generate 7-day future predictions using `scripts/run_prediction.py`:
- Use the last sequence to predict next 7 days
- Save predictions to `data/processed/future_predictions.csv`
- Visualize future predictions with historical context

# Future Predictions: Generate 7-day forecast
try:
    log("Generating future predictions...")
    
    # Get last sequence for prediction
    last_sequence = preprocessed_data.values[-SEQ_LEN:]
    target_col_idx = list(preprocessed_data.columns).index("Gold")
    
    # Generate future predictions
    future_predictions, future_dates = predict_future(
        model,
        feature_scaler,
        last_sequence,
        n_days=7,
        target_col_idx=target_col_idx
    )
    
    # Create DataFrame and save
    future_df = pd.DataFrame({
        "Date": future_dates,
        "Predicted_Gold_Price": future_predictions
    })
    future_df.to_csv(FUTURE_PREDICTIONS_PATH, index=False)
    log(f"Future predictions saved to {FUTURE_PREDICTIONS_PATH}")
    
    # Display predictions
    display(future_df)
    
    # Plot with historical context
    plot_future_predictions(
        test_dates, y_test_true_inv,
        future_dates, future_predictions,
        n_history=200,
        output_path=DATA_PROCESSED_PLOTS_DIR / "future_predictions.png"
    )
    
    log("Future predictions complete!")
    
except Exception as e:
    log(f"Future prediction failed: {e}")
    raise


In [ ]:
## Summary and Next Steps

This notebook has successfully:
- ✅ Collected raw data and saved to `data/raw/gold_raw_data.csv`
- ✅ Preprocessed data with feature engineering and saved to `data/processed/`
- ✅ Trained LSTM model and saved with timestamp to `models/`
- ✅ Evaluated model and saved results to `models/evaluation_results.csv`
- ✅ Created visualizations and saved to `data/processed/plots/`
- ✅ Generated 7-day future predictions and saved to `data/processed/future_predictions.csv`

**Consider improving the model by:**
- Adding exogenous feature engineering (macro surprises, term spreads, commodity spreads)
- Testing alternative sequence lengths, architectures (GRU, TCN), and hyperparameters
- Using walk-forward validation for more realistic evaluation
- Calibrating prediction intervals with quantile models or bootstrapping
- Incorporating regime detection (volatility states) for conditional modeling